# **Challenge - Alkemy**

# **Parte 2 - Requerimiento Funcional - Procesamiento de Datos**
1) Abrir las tablas que se descargaron en el punto anterior (Paso 1)
2) Instalamos e importamos librerías
3) Definimos las funciones a utilizar
4) Ejecutamos

## **1) Instalamos librerias**

In [89]:
# !pip install chardet

## **2) Importamos librerías a utilizar:**

In [90]:
import os     # recorrer carpetas
import glob   # para buscar el ultimo archivo modificado
import pandas as pd
import logging
import chardet
import numpy as np
# import requests, re
from datetime import datetime

# # Seteamos el idioma español para las fechas (funciones: fecha_ultimo_cambio y file_path)
# import locale
# locale.setlocale(locale.LC_ALL,'es_ES.UTF-8')

In [91]:
# os.getcwd()

## **3) Funciones a Utilizar**

In [92]:
# def fecha_ultimo_cambio(url_pagina):
#     ''' Obtiene la fecha de última modificación de los datos.
#     Buscando en la página la frase "Último cambio" y luego limpiando
#     la cadena de caracteres para obtener sólo la fecha (DD/MM/YYYY).
    
#     INPUT: url_pagina: pagina web donde se encuentra la fecha deseada
#     OUPUT: ultimo_cambio: fecha en formato: "16 julio 2018"   --------- FALTA ARREGLAR EL FORMATO PARA AGREGARLA A LA TABLA DE DATOS ------------
#     '''
    
#     pagina = requests.get(url = url_pagina)
#     requests_get_OK(pagina)
    
#     meses = ['enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio',
#              'julio', 'agosto', 'septiembre', 'setiembre', 'octubre',
#              'noviembre', 'diciembre']
    
#     # Busca y formatea la fecha
#     target = 'Último cambio'
#     f = re.search(target, pagina.text)
#     fecha_string = pagina.text[f.span()[0]+100:f.span()[1]+210]
#     ultimo_cambio = fecha_string.strip()    # Elimina los espacios en blancos que hay antes y después de la fecha
    
#     a = [True for m in meses if m in fecha_string.lower()]
#     if not a: 
#         print('No se encontró fecha de última actualización en la web')
#         logging.warning('No se encontró fecha de última actualización en la web')
#     else:
#         logging.info(f'Se encontró la fecha {ultimo_cambio}')
    
#     if 'de' in ultimo_cambio: 
#         ultimo_cambio = ultimo_cambio.replace(' de', '')
    
#     fecha = datetime.strptime(ultimo_cambio, '%d %B %Y')
#     ultima_fecha = fecha.strftime('%d/%m/%Y')
    
#     logging.info(f'Última modificación de la tabla: {ultima_fecha}')
    
#     return ultima_fecha

In [93]:
def buscar_archivos(carpeta_ppal = 'Cultura_dataset'):
    ''' Busca la carpeta principal donde se guardaron los archivos y
    busca los archivos con los datos, i.e. las tablas
    (Los archivos fuentes los habíamos guardado dentro de la carpeta "Cultura_dataset")
    Deben haber sólo 3 carpetas dentro de "Cultura_dataset", correspondientes a las
    tres categorías (i.e. bibliotecas populares, museos y cines).
    
    INPUT: carpeta principal, creada en la función file_path
    OUTPUT: lista con los path de cada archivo a procesar
    '''
    # Buscamos la carpeta raíz donde están las carpetas con los datos
    for root, dir, files in os.walk(os.getcwd()):
        if carpeta_ppal in dir:
            path = os.path.join(root, carpeta_ppal) 
#     print(f'El path de la carpeta {carpeta_ppal} es: {path}')
    
    p = path+'/*'
    # Carpetas de cada categoria
    folder_cat = glob.glob(p)
    
    # Selecciona las carpetas más nuevas de cada categoria
    last_folder = [max(glob.glob(ind+'/*'), key=os.path.getmtime) for ind in folder_cat if os.path.isdir(ind)]
    
    # Selecciona los archivos más nuevos de cada categoria
    last_files = [max(glob.glob(ind+'/*'), key=os.path.getmtime) for ind in last_folder]
        
    logging.info(f'Carpeta y archivos encontrados.')

    return last_files

**VER SI ME ELIGE LA CARPETA Y EL ARCHIVO MÁS NUEVO, EL MÁS ACTUALIZADO**

**Encoding y abrir archivo en un DataFrame**

In [94]:
def leer_archivos_DF(file):
    '''
    Chequeamos el encoding del archivo y lo abrimos.
    
    INPUT: el path del archivo a leer
    OUTPUT: DataFrame con la tabla de datos
    '''
    
    with open(file, 'rb') as data:
        result = chardet.detect(data.read(1000000))
    encode = result['encoding']
    
    df = pd.read_csv(file, encoding = encode, dtype=str)  
    # le pongo dtype=str, así de esta forma cdo tengo q juntar las columnas del teléfono, es igual para las 3 categorías.
    # si no lo pongo, cdo abro museos sale una forma distinta...
    logging.info(f'Archivo leído y listo para procesar.')

    
    return df

**Cambiamos la cadena 's/d' de la tabla por NaN**

(Enunciado PDF dice: "Los registros para los cuales la fuente no brinda información deben cargarse como nulos."")

In [95]:
def reemplazo_sd_NaN(df):
    '''
    Las tablas suelen tener registros nulos y tienen como valor 's/d' (sin datos).
    Chequeamos si tienen y los reemplazamos por el valor nulo: np.NaN
    
    INPUT: el dataframe de la tabla de datos
    OUTPUT: tabla de datos resultante
    '''
    if 's/d' in df.values:
        df = df.replace('s/d',np.NaN)
    return df

**Estandarizamos los nombres de las Provincias**

In [96]:
def corregir_provincias(dataframe, col_name):
    '''
    Debido a que a veces encontramos el nombre de las provincias
    sin tildes o en minúsculas, buscamos que las provincias
    estén escritas de una sóla forma, igual en todas las tablas.
    
    INPUT: dataframe: tabla con los datos (tipo dataframe de pandas)
           col_name: nombre de la columna que contiene a las provincias
    OUTPUT: dataframe estandarizado
    '''
    # Así quiero que salgan los nombres de las provincias
    provincias = ['San Juan', 'Ciudad Autonoma de Buenos Aires',
                  'Buenos Aires', 'Entre Ríos','Santa Fé', 'Corrientes',
                  'Córdoba', 'San Luis', 'Santiago del Estero','Tucumán',
                  'Mendoza', 'La Rioja', 'Catamarca', 'Salta', 'Jujuy',
                  'Chaco', 'Formosa', 'Misiones', 'La Pampa', 'Neuquén',
                  'Río Negro', 'Santa Cruz','Tierra del Fuego', 'Chubut']
    
    # Transformo las vocales con tildes por vocales sin tildes
    # y después paso las provincias a minúsculas.
    a = 'áéíóú'
    b = 'aeiou'
    trans = str.maketrans(a,b)
    
    dataframe[col_name] = dataframe[col_name].str.translate(trans).str.lower()
    
    for p in provincias:
        mask = p.translate(trans).lower() 
        dataframe.loc[dataframe[col_name].str.contains(mask), col_name] = p
    
    return dataframe   

## **4) Ejecutamos**

- **suponemos que los códigos de área de los números de teléfonos están bien**
(cosa que no pasa, al menos en cines)

In [ ]:
# ''' URLs dadas: No es la url directa a la tabla '''
# url_museos = 'https://datos.gob.ar/dataset/cultura-mapa-cultural-espacios-culturales/archivo/cultura_4207def0-2ff7-41d5-9095-d42ae8207a5d'
# url_cines = 'https://datos.gob.ar/dataset/cultura-mapa-cultural-espacios-culturales/archivo/cultura_392ce1a8-ef11-4776-b280-6f1c7fae16ae'
# url_bibliotecas = 'https://datos.gob.ar/dataset/cultura-mapa-cultural-espacios-culturales/archivo/cultura_01c6c048-dbeb-44e0-8efa-6944f73715d7'

In [99]:
# '''Buscamos la última fecha de modificación de las tablas '''
# ### hay una función que se usa en este archivo (2da parte) y en el primero (1ra parte)

# museo_fecha_act = fecha_ultimo_cambio(url_museos)  # da la fecha como string
# cines_fecha_act = fecha_ultimo_cambio(url_cines)
# biblio_fecha_act = fecha_ultimo_cambio(url_bibliotecas)

# museo_fecha_act, cines_fecha_act, biblio_fecha_act

In [ ]:
# Usamos el archivo info.LOG creado anteriormente
logging.basicConfig(filename= 'info.log', format='%(asctime)s : %(levelname)s : %(message)s',
                    datefmt='%d/%m/%y %H:%M:%S', level=logging.INFO,
                    filemode = 'a', force=True)
logging.info('EJECUTANDO LA PARTE 2 (2do archivo....')

In [101]:
# Archivos a procesar:
archivos = buscar_archivos()
#archivos  # lista con los archivos a utilizar

In [ ]:
# Ver la fecha de última modifiación
#print(datetime.fromtimestamp(os.path.getmtime(archivos[0])))

### **Abrimos archivos y reemplazamos 's/d' por np.NaN**

In [103]:
# Asignamos nombres a los archivos
biblio = archivos[0]
museos = archivos[1]
cines = archivos[2]

# Abrimos los archivos en formato DataFrame y dtype=string
biblio_df = leer_archivos_DF(biblio)
museos_df = leer_archivos_DF(museos)
cines_df = leer_archivos_DF(cines)

# Reemplazamos los 's/d' por np.NaN
biblio_df_nan = reemplazo_sd_NaN(biblio_df)
museos_df_nan = reemplazo_sd_NaN(museos_df)
cines_df_nan = reemplazo_sd_NaN(cines_df)

### **Modificamos los nombres de las columnas**

In [105]:
# Nombre de las columnas dadas en el PDf del Challenge, a utilizar
columnas = ['Cod_localidad', 'Id_provincia', 'Id_departamento',
                'Categoría', 'Provincia', 'Localidad', 'Nombre', 
                'Domicilio', 'Código postal', 'Número de teléfono',
                'Email', 'Web', 'Fuente']

In [107]:
## Renombramos algunas columnas, según fueron dadas en el enunciado del challenge.
## Lo hago categoría por categoría porque hay diferencias en nombres y hasta columnas que faltan

'''  Bibliotecas Populares '''
biblio_df_nan.rename(columns={'Cod_Loc':columnas[0], 'IdProvincia':columnas[1],
                              'IdDepartamento':columnas[2], 'Categoría':columnas[3],
                              'Provincia':columnas[4], 'Localidad':columnas[5], 
                              'Nombre':columnas[6], 'Domicilio':columnas[7],
                              'CP':columnas[8], 'Teléfono':columnas[9],
                              'Mail':columnas[10], 'Web':columnas[11]},
                     inplace=True)
    
'''  Museos '''
museos_df_nan.rename(columns={'localidad_id':columnas[0], 'provincia_id':columnas[1],
                              'provincia':columnas[4], 'localidad':columnas[5], 
                              'nombre':columnas[6], 'direccion':columnas[7],
                              'codigo_postal':columnas[8], 'telefono':columnas[9],
                              'mail':columnas[10], 'web':columnas[11], 'fuente':columnas[12]},
                     inplace=True)

# Agregamos la columna 'Id_departamento' a partir del Código de Localidad.
for ind in range(len(museos_df_nan)):
    museos_df_nan.loc[ind, 'Id_departamento'] = int(str(museos_df_nan['Cod_localidad'][ind])[:-3])

# Defino la columna tipo Integer
museos_df_nan['Id_departamento'] = museos_df_nan['Id_departamento'].astype('Int64')

# Agregamos la columna 'Categoría' que en este caso es 'Museos'
museos_df_nan['Categoría'] = 'Museos'

'''  Cines '''
cines_df_nan.rename(columns={'Cod_Loc':columnas[0], 'IdProvincia':columnas[1],
                             'IdDepartamento':columnas[2], 'Categoría':columnas[3],
                             'Provincia':columnas[4], 'Localidad':columnas[5], 
                             'Nombre':columnas[6], 'Dirección':columnas[7],
                             'CP':columnas[8], 'Teléfono':columnas[9],
                             'Mail':columnas[10], 'Web':columnas[11]},
                    inplace=True)

### **Modificamos y estandarizamos los nombres de las Provincias**

Nuestro país está dividido en 24 jurisdicciones: 23 provincias y un distrito federal (Ciudad Autónoma de Buenos Aires).

In [109]:
biblio_df_nan = corregir_provincias(biblio_df_nan, 'Provincia')
museos_df_nan = corregir_provincias(museos_df_nan, 'Provincia')
cines_df_nan = corregir_provincias(cines_df_nan, 'Provincia')

### **Unimos el código de área con el número de teléfono**

Pusimos en el read_csv, dtype=string, para que no hubiera problemas al unir el codigo de telefono con el numero de telefono.
Lo único es q habrán teléfonos q empiezan con 011 y otros con 11, por como fueron ingresados.

In [ ]:
# No hago función porq es una línea para c/tabla
biblio_df_nan['Número de teléfono'] = biblio_df_nan['Cod_tel'] + biblio_df_nan['Número de teléfono']  # queda 11 en lugar de 011
museos_df_nan['Número de teléfono'] = museos_df_nan['codigo_indicativo_telefono'] + museos_df_nan['Número de teléfono']  # queda el 011 en lugar de 11
cines_df_nan['Número de teléfono'] = cines_df_nan['cod_area'] + cines_df_nan['Número de teléfono']  # queda 11 en lugar de 011

### **Creamos tabla con información sobre los Cines**

In [135]:
''' La columna "espacio_INCAA" tiene varios valores, no está estandarizado
entonces vamos a reemplazar los "Si" por 1, y el resto por 0 (que serían los "No"),
para que queden sólo 2 valores '''

cines_df_nan['espacio_INCAA'].replace(['si','Si','SI', '1'], 1, inplace=True)
cines_df_nan['espacio_INCAA'].replace(['nan','0',np.NaN, 'no', 'NO', 'No'], 0, inplace=True)
if len(cines_df_nan['espacio_INCAA'].unique()) != 2: 
    print('Error en la estandarización INCAA')  # verificar q sólo hay 1 y 0
    print('Verificar que valor no se modificó por 0 o 1')
    print(cines_df_nan['espacio_INCAA'].unique())


[0 1]


In [111]:
''' Cambiamos el dtype de algunas columnas para crear esta tabla.
 Los registros NaN, dan error si elijo dtype=integer, y si elijo dtype=string cambia a <NA> '''

cols = ['Provincia', 'Pantallas', 'Butacas', 'espacio_INCAA']
cines_INCAA = cines_df_nan.astype({'Pantallas':'int', 'Butacas':'int', 'espacio_INCAA':'int'})

In [112]:
# Agrupamos por provincia y calculamos la cantidad de Pantallas, Butacas y espacios INCAA que hay.
tabla_INCAA = cines_INCAA[cols].groupby(by='Provincia').sum().sort_values(by='Pantallas', ascending=False)

In [113]:
# Guardamos la tabla como un archivo .csv
path_incaa = os.path.normpath(os.path.join(cines, "..", "..", "..")) + '/cines_INCAA.csv'
tabla_INCAA.to_csv(path_incaa, sep=',')

print(f'se guardó la tabla de espacios INCAA en {path_incaa}')
logging.info(f'se guardó la tabla de espacios INCAA en {path_incaa}')

se guardó la tabla de espacios INCAA en /home/leono/prueba/cultura_argentina/Cultura_dataset/cines_INCAA.csv


### **Eliminamos las columnas que no vamos a utilizar**

In [114]:
biblio_final = biblio_df_nan.drop(columns=[col for col in biblio_df_nan if col.capitalize() not in columnas])
museos_final = museos_df_nan.drop(columns=[col for col in museos_df_nan if col.capitalize() not in columnas])
cines_final = cines_df_nan.drop(columns=[col for col in cines_df_nan if col.capitalize() not in columnas])

### **Agregramos la fecha de última modificación de la tabla en la columna 'Ultima_actualización'**  --  No lo hago acá, veremos si lo hacemos cdo usamos SQL

In [115]:
# biblio_final['Ultima_actualización'] = biblio_fecha_act
# museos_final['Ultima_actualización'] = museo_fecha_act
# cines_final['Ultima_actualización'] = cines_fecha_act

### **Creamos y guardamos una única tabla**

In [136]:
tablas = [biblio_final, museos_final, cines_final]
tabla_final = pd.concat(tablas, ignore_index=True)
#tabla_final.columns

In [137]:
path_tabla = os.path.normpath(os.path.join(cines, "..", "..", "..")) + '/tabla_unica.csv'

# Eliminamos la columna 'Fuente', que no es necesaria en esta tabla
tabla_unica = tabla_final.drop(columns = 'Fuente')
tabla_unica.to_csv(path_tabla, sep=',')

print(f'Se guardó la tabla que convina las 3 categorías en {path_tabla}')
logging.info(f'se guardó la tabla que convina las 3 categorías en {path_tabla}')

Se guardó la tabla que convina las 3 categorías en /home/leono/prueba/cultura_argentina/Cultura_dataset/tabla_unica.csv


### **Creamos una tabla con las siguientes sub-tablas:**
 1) Cantidad de registros totales por categoría
 2) Cantidad de registros totales por fuente
 3) Cantidad de registros por provincia y categoría
 
 usando la tabla final (que tiene la columna 'Fuente')

In [118]:
# path donde se guardará las tablas
path_final = os.path.normpath(os.path.join(cines, "..", "..", "..")) + '/datos_conjuntos.csv'

In [119]:
''' Tabla 1: Cantidad de registros totales por categoría '''
reg_por_categoria = tabla_final['Categoría'].value_counts()

In [120]:
''' Tabla 2: Cantidad de registros totales por fuente '''
reg_por_fuente = tabla_final['Fuente'].value_counts()

In [121]:
''' Tabla 3: Cantidad de registros por Provincia y Categoría '''
reg_prov_categ = tabla_final.groupby(by=['Provincia', 'Categoría']).size()

In [139]:
''' Guardamos las tres tablas anteriores en un mismo archivo 
   (según entiendo del enunciado) '''
with open(path_final, 'w') as f:
    f.write('# Cantidad de registros totales por categoría:')
    reg_por_categoria.to_csv(f, header = False, mode = 'a')
    f.write('\n')
    f.write('# Cantidad de registros totales por fuente:')
    reg_por_fuente.to_csv(f, header = False, mode = 'a')
    f.write('\n')
    f.write('# Cantidad de registros por provincia y por categoría:')
    reg_prov_categ.to_csv(f, header = False, mode = 'a')

print(f'se guardaron las 3 tablas juntas en  {path_final}')
logging.info(f'se guardaron las 3 tablas juntas en  {path_final}')

se guardaron las 3 tablas juntas en  /home/leono/prueba/cultura_argentina/Cultura_dataset/mas_datos.csv


In [140]:
logging.info('.........Terminó el Procesado de Datos (2da parte)')
print(' \n    Terminó el Procesado de Datos (2da parte)')

 
    Terminó el Procesado de Datos (2da parte)


## **PRUEBAS**

In [124]:
# biblio['Número de teléfono'] = biblio['Cod_tel'] + biblio['Número de teléfono']
#     # Correr una sóla vez, xq sino da error si se corre por segunda vez sobre la Serie ya modificada
# museos['Número de teléfono'] = museos['codigo_indicativo_telefono'].astype('Int64').astype(str) + museos['Número de teléfono'].astype('Int64').astype(str)
# museos['Número de teléfono'] = museos['Número de teléfono'].replace('<NA><NA>', 'nan')
# museos['Número de teléfono']

### Unimos Domicilio con el piso ( si lo tiene)

In [125]:
# # Unimos el Domicilio con el Piso (si lo tiene)
# pp = cines['Piso'].astype(str)

# for idx in range(len(cines)):
#     if 'nan' in pp[idx]:
#         cines.loc[idx, 'Domicilio'] = cines['Domicilio'][idx]
#     else:
#         cines.loc[idx, 'Domicilio'] = cines['Domicilio'][idx] + ', ' + pp[idx]

### Modificamos el código postal para que sea sólo números

In [126]:
# if cines_col['Código Postal'].dtype != int:
#     cines_col['Código Postal'] = cines_col['Código Postal'].str.replace(r"[a-zA-Z]", '', regex=True)

In [127]:
# if museos_col['Código Postal'].dtype != int:
#     museos_col['Código Postal'] = museos_col['Código Postal'].str.replace(r"[a-zA-Z]", '', regex=True)

### en MUSEOS 
2) Analizamos si hay relación entre los Cod_localidad y los Id_departamento en las tablas biblioteca y cines

In [128]:
# b = biblio[['Cod_localidad', 'Id_provincia', 'Id_departamento', 'Provincia', 'Departamento', 'Localidad']]
# # print(b.columns)

# # Probemos si los 'Id_departamento' son iguales al 'Cod_localidad', sacandole los últimos 3 digitos a éste último.
# for i in range(len(b)):
#     if int(str(b['Cod_localidad'][i])[:-3]) != b['Id_departamento'][i]:
#         print('Error en la sgte línea: ', i, b['Cod_localidad'][i], b['Id_departamento'][i], b['Departamento'][i], b['Localidad'][i] )

In [129]:
# c = cines[['Cod_localidad', 'Id_provincia', 'Id_departamento', 'Provincia', 'Departamento', 'Localidad']]
# # print(c.columns)

# # Probemos si los 'Id_departamento' son iguales al 'Cod_localidad', sacandole los últimos 3 digitos a éste último.
# for i in range(len(c)):
#     if int(str(c['Cod_localidad'][i])[:-3]) != c['Id_departamento'][i]:
#         print('Error en la sgte línea: ', i, c['Cod_localidad'][i], c['Id_departamento'][i], c['Departamento'][i], c['Localidad'][i] )

Se ve que sl Cod_localidad e Id_departamento están relacionados. 

- Se ve que si le sacamos los últimos 3 digitos al Cod_localidad obtenemos el Id_departamento
- En la tabla cines se encuentran 2 errores en las filas: 70 y 304. Parece q está mal departamento de Haedo y El Palomar y por lo tanto el id_dpto.
 - Link a la tabla de datos del INDEC sobre códigos de localidades 
https://redatam.indec.gob.ar/redarg/CENSOS/CPV2010rad/Docs/codigos_provincias.pdf?_ga=2.242401410.1865600674.1640183183-1178966641.1640183183

3) Ahora comparemos la tabla biblioteca con la museos. 

In [130]:
# m = museos[['espacio_cultural_id', 'Id_provincia', 'Cod_localidad', 'Provincia', 'Localidad']]

In [131]:
# # Fusionemos la tabla 'biblioteca' y 'museos'
# pd.merge(b,m, on=['Provincia','Id_provincia','Localidad']).head(5)

 Parece correcto que el codigo de localidad sin los últimos 3 digitos sea el id del departamnto !!!

 4) Agregamos la columna Categoría e Id_departamento a la tabla Museos

In [132]:
# # Agregamos la columna 'Id_departamento' a partir del Código de Localidad.
# for ind in range(len(museos)):
#     museos.loc[ind, 'Id_departamento'] = int(str(museos['Cod_localidad'][ind])[:-3])

# museos['Id_departamento'] = museos['Id_departamento'].astype('Int64')

# # # Agregamos la columna 'Categoría' = Museos
# museos['Categoría'] = 'Museos'
# type(museos)

In [133]:
# museos.columns